In [1]:
from brian2 import *
from keras.datasets import mnist
from tqdm import tqdm
from sklearn import metrics
import numpy as np
import pandas as pd
import itertools
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix
from spikingjelly import visualizing
import utils
from collections import defaultdict
import networkx as nx
from matplotlib.ticker import MultipleLocator
from sklearn.model_selection import train_test_split
import os
import seaborn as sns

In [2]:
(X_train, y_train), (X_test, y_test) = mnist.load_data()

# simplified classification (0 1 and 8)
X_train = X_train[(y_train == 1) | (y_train == 0)]
y_train = y_train[(y_train == 1) | (y_train == 0)]
X_test = X_test[(y_test == 1) | (y_test == 0)]
y_test = y_test[(y_test == 1) | (y_test == 0)]

# pixel intensity to Hz (255 becoms ~63Hz)
X_train = X_train / 4 
X_test = X_test / 4

# Объединяем обучающие и тестовые данные для стратификации

X = np.concatenate((X_train, X_test), axis=0)
y = np.concatenate((y_train, y_test), axis=0)

# Стратифицированное разбиение для обучающего набора
X_train, X_temp, y_train, y_temp = train_test_split(
    X, y, test_size=(len(y) - 2), stratify=y)

# Стратифицированное разбиение для тестового набора
X_test, _, y_test, _ = train_test_split(
    X_temp, y_temp, test_size=(len(y_temp) - 2), stratify=y_temp)

X_eval, _, y_eval, _ = train_test_split(
    X_temp, y_temp, test_size=(len(y_temp) - 2), stratify=y_temp)

X_train.shape, X_test.shape

((2, 28, 28), (2, 28, 28))

In [3]:
from collections import Counter

# Используем Counter для подсчета количества каждой метки
value_list = y_train.tolist()
label_counts = Counter(value_list)

# Выводим результат
print(label_counts)

Counter({0: 1, 1: 1})


In [4]:
def generate_small_world(n_nodes, k, p):
    return nx.watts_strogatz_graph(n_nodes, k, p)

In [10]:
n_input = 28 * 28  # input layer
n_e = 100  # e - excitatory
n_i = n_e  # i - inhibitory

v_rest_e = -60. * mV  # v - membrane potential
v_reset_e = -65. * mV
v_thresh_e = -52. * mV

v_rest_i = -60. * mV
v_reset_i = -45. * mV
v_thresh_i = -40. * mV

taupre = 20 * ms
taupost = taupre
gmax = .3  #.01
gmax_small = .3
dApre = .01
dApost = -dApre * taupre / taupost * 1.05
dApost *= gmax
dApre *= gmax

# Apre and Apost - presynaptic and postsynaptic traces, lr - learning rate
stdp = '''w : 1
    lr : 1 (shared)
    dApre/dt = -Apre / taupre : 1 (event-driven)
    dApost/dt = -Apost / taupost : 1 (event-driven)'''
pre = '''ge += w
    Apre += dApre
    w = clip(w + lr*Apost, 0, gmax)'''
post = '''Apost += dApost
    w = clip(w + lr*Apre, 0, gmax)'''


class Model():

    def __init__(self, debug=False):
        app = {}
        
        self.g = generate_small_world(n_e, 20, 0.5)
        
        # input images as rate encoded Poisson generators
        app['PG'] = PoissonGroup(n_input, rates=np.zeros(n_input) * Hz, name='PG')

        # excitatory group
        neuron_e = '''
            dv/dt = (ge*(0*mV-v) + gi*(-100*mV-v) + (v_rest_e-v)) / (100*ms) : volt
            dge/dt = -ge / (5*ms) : 1
            dgi/dt = -gi / (10*ms) : 1
            '''
        app['EG'] = NeuronGroup(n_e, neuron_e, threshold='v>v_thresh_e', refractory=5 * ms, reset='v=v_reset_e',
                                method='euler', name='EG')
        app['EG'].v = v_rest_e - 20. * mV

        if (debug):
            app['ESP'] = SpikeMonitor(app['EG'], name='ESP')
            app['ESM'] = StateMonitor(app['EG'], ['v'], record=True, name='ESM')
            app['ERM'] = PopulationRateMonitor(app['EG'], name='ERM')

        # ibhibitory group
        neuron_i = '''
            dv/dt = (ge*(0*mV-v) + (v_rest_i-v)) / (10*ms) : volt
            dge/dt = -ge / (5*ms) : 1
            '''
        app['IG'] = NeuronGroup(n_i, neuron_i, threshold='v>v_thresh_i', refractory=2 * ms, reset='v=v_reset_i',
                                method='euler', name='IG')
        app['IG'].v = v_rest_i - 20. * mV

        if (debug):
            app['ISP'] = SpikeMonitor(app['IG'], name='ISP')
            app['ISM'] = StateMonitor(app['IG'], ['v'], record=True, name='ISM')
            app['IRM'] = PopulationRateMonitor(app['IG'], name='IRM')

        # poisson generators one-to-all excitatory neurons with plastic connections 
        app['S1'] = Synapses(app['PG'], app['EG'], stdp, on_pre=pre, on_post=post, method='euler', name='S1')
        app['S1'].connect()
        app['S1'].w = 'rand()*gmax'  # random weights initialisation
        app['S1'].lr = 1  # enable stdp     
        
        app['S_small_world'] = Synapses(app['EG'], app['EG'], stdp, on_pre=pre, on_post=post, method='euler', name='S_small_world')
        
        for (u, v) in self.g.edges():
            print(f'{u}---{v}')
            app['S_small_world'].connect(j=u, i=v)
            app['S_small_world'].w = 'rand()*gmax_small'  # здесь нужно сделать так, чтобы генерировались
            # рандомные веса на порядок меньше, чем основные
        app['S_small_world'].lr = 1
        
        self.res_0 = [0]
        for g, v in self.g.edges():
            if g == 0:
                self.res_0.append(v)

        if (debug):
            # some synapses
            app['S1M'] = StateMonitor(app['S1'], ['w', 'Apre', 'Apost'], record=app['S1'][380, :4], name='S1M')

            # excitatory neurons one-to-one inhibitory neurons
        app['S2'] = Synapses(app['EG'], app['IG'], 'w : 1', on_pre='ge += w', name='S2')
        app['S2'].connect(j='i')
        app['S2'].delay = 'rand()*10*ms'
        app['S2'].w = 3  # very strong fixed weights to ensure corresponding inhibitory neuron will always fire

        # inhibitory neurons one-to-all-except-one excitatory neurons
        app['S3'] = Synapses(app['IG'], app['EG'], 'w : 1', on_pre='gi += w', name='S3')
        app['S3'].connect(condition='i!=j')
        app['S3'].delay = 'rand()*5*ms'
        app[
            'S3'].w = .03  # weights are selected in such a way as to maintain a balance between excitation and ibhibition

        self.net = Network(app.values())
        self.net.run(0 * second)

    def __getitem__(self, key):
        return self.net[key]

    def train(self, X, y,  epoch=1, draw=False):
        self.net['S1'].lr = 1 
        self.net['S_small_world'].lr = 1 # stdp on

        for ep in range(epoch):
            for idx in tqdm(range(len(X))):
                
                mon = SpikeMonitor(self.net['EG'], name='TestRM')
                mon2 = SpikeMonitor(self.net['IG'], name='TestIM')
    
                self.net.add(mon)
                self.net.add(mon2)
                
                # active mode
                self.net['PG'].rates = X[idx].ravel() * Hz
                self.net.run(0.35 * second)

                # passive mode
                self.net['PG'].rates = np.zeros(n_input) * Hz
                self.net.run(0.15 * second)
                
                if draw:
                    # plot_all_spikes(mon, f'spikes/test/spikes_all_{idx}_class_{y[idx]}.png')
                    plot_same_groups(mon, f'spikes/epoch_{ep}_spikes_{idx}_class_{y[idx]}.png', 0.25)
                    generate_df_from_spikes(mon, f'spikes/epoch_{ep}_spikes_{idx}_class_{y[idx]}.txt')
                
                ''' if y[idx] == 0: # сохранять только каждую 500 итерацию
                    # plot_same_spikes(mon, f'spikes/spikes_{idx}.png', 0.05)
                    # plot_same_groups(mon, f'spikes/spikes_{idx}.png', 0.05)
                    
                    if len(os.listdir('spikes')) == 0:
                        
                        spike_times = mon.t / ms
                        neuron_indices = mon.i
                        
                        # Создание словаря для хранения индексов нейронов и их времени спайков
                        neuron_data = {neuron: time for neuron, time in zip(neuron_indices, spike_times)}
                        
                        # Создание групп нейронов на основе условия разницы времени спайков меньше 50 мс
                        current_group = []
                        prev_time = spike_times[0]
                        max_group = []
                        
                        for neuron, time in neuron_data.items():
                            if abs(time - prev_time) < 0.25:
                                current_group.append(neuron)
                                
                            else:
                                if len(current_group) > len(max_group):
                                    max_group = current_group.copy()
                                current_group = [neuron]
                                prev_time = time
                        
                        if len(current_group) > len(max_group):
                            max_group = current_group.copy()
                            
                        print(sort(max_group))
                        plt.rcParams["figure.figsize"] = (20, 6)
                        for neuron in max_group:
                            scatter(neuron_data[neuron], neuron, color='r')
                        
                        xlabel('Время спайка')
                        ylabel('Индексы нейронов')
                        title('Точечный график с окрашиванием самой большой группы нейронов')
                        grid(True)
                        savefig(f'spikes/spikes_{idx}.png')
                        close()
                        
                        plt.rcParams["figure.figsize"] = (20, 6)
                        spike_dict = defaultdict(list)
                        for time, neuron in zip(mon.t / ms, mon.i):
                            spike_dict[time].append(neuron)
                        # Находим самую большую группу одновременно спайкующих нейронов
                        max_group = max(spike_dict.values(), key=len)
                        max_time = max(spike_dict, key=lambda k: len(spike_dict[k]))
                        
                        # Формируем данные для построения графика для нейронов из самой большой группы
                        max_group_spike_times = []
                        max_group_neuron_indices = []
                        for time, neurons in spike_dict.items():
                            for neuron in neurons:
                                if abs(time - max_time) <= 0.05 or neuron in max_group:
                                    max_group_spike_times.append(time)
                                    max_group_neuron_indices.append(neuron)
     
                        scatter(max_group_spike_times, max_group_neuron_indices, color='red', label='Самая большая группа')
                        ylabel('Neuron index')
                        legend()
                        savefig(f'spikes/spikes_{idx}.png')
                        close()
                    else:
                        plot_spikes(mon, None, f'spikes/spikes_{idx}.png', color_neurons=max_group)'''
                        
                                
                self.net.remove(self.net['TestRM'])
                self.net.remove(self.net['TestIM'])

    def evaluate(self, X, y, draw=False):
        self.net['S1'].lr = 0
        self.net['S_small_world'].lr = 0# stdp off

        features = []
        for idx in tqdm(range(len(X))):
            # rate monitor to count spikes
            mon = SpikeMonitor(self.net['EG'], name='RM')
            self.net.add(mon)

            # active mode
            self.net['PG'].rates = X[idx].ravel() * Hz
            self.net.run(0.35 * second)

            # spikes per neuron foreach image
            features.append(np.array(mon.count, dtype=int8))
                
            # passive mode
            self.net['PG'].rates = np.zeros(n_input) * Hz
            self.net.run(0.15 * second)
            
            if draw:
                # plot_all_spikes(mon, f'spikes/test/spikes_all_{idx}_class_{y[idx]}.png')
                plot_same_groups(mon, f'spikes/test/spikes_{idx}_class_{y[idx]}.png', 0.25)
                generate_df_from_spikes(mon, f'spikes/test/spikes_{idx}_class_{y[idx]}.txt')

            self.net.remove(self.net['RM'])

        return features


In [11]:
def plot_w(S1M):
    plt.rcParams["figure.figsize"] = (20,10)
    subplot(311)
    plot(S1M.t/ms, S1M.w.T/gmax)
    ylabel('w / wmax')
    subplot(312)
    plot(S1M.t/ms, S1M.Apre.T)
    ylabel('apre')
    subplot(313)
    plot(S1M.t/ms, S1M.Apost.T)
    ylabel('apost')
    tight_layout()
    show();
    
def plot_v(ESM, ISM, neuron=13):
    plt.rcParams["figure.figsize"] = (20,6)
    cnt = -50000 # tail
    plot(ESM.t[cnt:]/ms, ESM.v[neuron][cnt:]/mV, label='exc', color='r')
    plot(ISM.t[cnt:]/ms, ISM.v[neuron][cnt:]/mV, label='inh', color='b')
    plt.axhline(y=v_thresh_e/mV, color='pink', label='v_thresh_e')
    plt.axhline(y=v_thresh_i/mV, color='silver', label='v_thresh_i')
    legend()
    ylabel('v')
    show();
    
def plot_rates(ERM, IRM):
    plt.rcParams["figure.figsize"] = (20,6)
    plot(ERM.t/ms, ERM.smooth_rate(window='flat', width=0.1*ms)*Hz, color='r')
    plot(IRM.t/ms, IRM.smooth_rate(window='flat', width=0.1*ms)*Hz, color='b')
    ylabel('Rate')
    show();
    
def plot_all_spikes(ESP, path):
    plt.rcParams["figure.figsize"] = (20, 6)
    scatter([spike_time for spike_time in ESP.t / ms if spike_time <= ESP.t[0] / ms + 100], 
            [neuron for neuron, time in zip(ESP.i, ESP.t / ms) if time <= ESP.t[0] / ms + 100], label='exc')
    ylabel('Neuron index')
    legend()
    savefig(path)
    close();
    
def plot_spikes(ESP, ISP, path, color_neurons=None):
    plt.rcParams["figure.figsize"] = (20, 6)
    #vplot(ESP.t / ms, ESP.i, '.b', label='exc')
    if ISP is not None:
        plot(ISP.t / ms, ISP.i, '.g', label='inh')
    positions = [index for index, value in enumerate(ESP.i) if value in color_neurons]
    spike_dict = defaultdict(list)
    for time, neuron in zip(ESP.t[positions] / ms, ESP.i[positions]):
        spike_dict[time].append(neuron)

    # Находим самую большую группу одновременно спайкующих нейронов
    max_group = max(spike_dict.values(), key=len)
    max_time = max(spike_dict, key=lambda k: len(spike_dict[k]))
    
    # Формируем данные для построения графика для нейронов из самой большой группы
    max_group_spike_times = []
    max_group_neuron_indices = []
    for time, neurons in spike_dict.items():
        for neuron in neurons:
            if abs(time - max_time) <= 0.25 or neuron in max_group:
                max_group_spike_times.append(time)
                max_group_neuron_indices.append(neuron)

    scatter(max_group_spike_times, max_group_neuron_indices, color='red', label='Самая большая группа')
    ylabel('Neuron index')
    legend()
    savefig(path)
    close();
    # plot(ESP.t[positions] / ms, ESP.i[positions], '.r', label='group')
    # ylabel('Neuron index')
    # legend()
    # savefig(path)
    # close();
    
def plot_same_groups(ESP, path_grahp, dt, path_txt=None):
    
    spike_times = [round(spike_time, 2) for spike_time in ESP.t / ms if spike_time <= ESP.t[0] / ms + 100]
    neuron_indices = [neuron for neuron, time in zip(ESP.i, ESP.t / ms) if time <= ESP.t[0] / ms + 100]
    
    neuron_spikes = list(zip(neuron_indices, spike_times))

    # Создание групп нейронов на основе условия разницы времени спайков меньше dt
    neuron_groups = []
    current_group = [neuron_spikes[0]]
    prev_neuron, prev_time = neuron_spikes[0]
    
    for neuron, time in neuron_spikes[1:]:
        if time - prev_time <= dt:
            current_group.append((neuron, time))
        else:
            neuron_groups.append(current_group)
            current_group = [(neuron, time)]
            prev_time = time
    
    neuron_groups.append(current_group)
    
    if path_txt:
        with open(path_txt, 'w') as file:
            for item in neuron_groups:
                file.write(f"{item}\n")
    
    # Создание цветовой метки для каждой группы нейронов
    colors_2 = ['grey', 'm']
    colors_3 = ['b', 'r']
    colors_10 = ['gold', 'pink']
    
    
    # Построение точечного графика с окрашиванием нейронов по группам
    fig, ax = plt.subplots(figsize=(15, 10))
    ax.set_ylabel('Индексы нейронов')
    ax.set_xlabel('Время спайка')
    # ax.title('Точечный график с окрашиванием нейронов по группам')
    ax.yaxis.set_major_locator(MultipleLocator(5))
    # ax.xaxis.set_major_locator(MultipleLocator(0.1))
    # ax.set_xticklabels(ax.get_xticks(), rotation=90)
    ax.grid(True)
    for i, group in enumerate(neuron_groups):
        
        color_2 = colors_2[i % len(colors_2)]
        color_3 = colors_3[i % len(colors_3)]
        color_10 = colors_10[i % len(colors_10)]
        
        if len(group)==1:
                for neuron, time in group:
                    plt.scatter(time, neuron, color='black', s=10, label=f'Нейрон {neuron}')
        if len(group)==2:
            for neuron, time in group:
                plt.scatter(time, neuron, color=color_2, s=10, label=f'Нейрон {neuron}')
        if len(group)>=3 and len(group)<10:
            for neuron, time in group:
                plt.scatter(time, neuron, color=color_3, s=10, label=f'Нейрон {neuron}')
        if len(group)>10:
            for neuron, time in group:
                plt.scatter(time, neuron, color=color_10, s=10, label=f'Нейрон {neuron}')

            
    fig.savefig(path_grahp)
    plt.close(fig)

    
def plot_same_spikes(ESP, path, dt):
    plt.rcParams["figure.figsize"] = (20, 6)
    
    spike_dict = defaultdict(list)
    for time, neuron in zip(ESP.t / ms, ESP.i):
        spike_dict[time].append(neuron)

    # Находим самую большую группу одновременно спайкующих нейронов
    max_group = max(spike_dict.values(), key=len)
    max_time = max(spike_dict, key=lambda k: len(spike_dict[k]))
    
    # Формируем данные для построения графика для нейронов из самой большой группы
    max_group_spike_times = []
    max_group_neuron_indices = []
    for time, neurons in spike_dict.items():
        for neuron in neurons:
            if abs(time - max_time) <= dt or neuron in max_group:
                max_group_spike_times.append(time)
                max_group_neuron_indices.append(neuron)

    scatter(max_group_spike_times, max_group_neuron_indices, color='red', label='Самая большая группа')
    ylabel('Neuron index')
    legend()
    savefig(path)
    close();


def test0(train_items=30):
    '''
    STDP visualisation
    '''
    seed(0)
    
    model = Model(debug=True)
    model.train(X_train[:train_items], epoch=1)
    
    plot_w(model['S1M'])
    plot_v(model['ESM'], model['ISM'])
    plot_rates(model['ERM'], model['IRM'])
    plot_spikes(model['ESP'], model['ISP'])
    
def generate_df_from_spikes(ESP, path, time=100):
    
    spike_times = [round(spike_time, 2) for spike_time in ESP.t / ms if spike_time <= ESP.t[0] / ms + time]
    neuron_indices = [neuron for neuron, time in zip(ESP.i, ESP.t / ms) if time <= ESP.t[0] / ms + time]

    # Создаем список словарей для каждой строки итогового DataFrame
    data = [{'time': time, 'index': neuron_index, 'spike': 1 if (time, neuron_index) in zip(spike_times, neuron_indices) else 0}
            for time in unique(spike_times) for neuron_index in range(101)]

    # Создаем DataFrame из списка словарей
    output_df = pd.DataFrame.from_records(data)
    
    # output_df.drop_duplicates(inplace=True)
    

    # Сохраняем DataFrame в текстовый файл
    output_df.to_csv(path, sep=' ')

0---99
0---98
0---3
0---4
0---5
0---94
0---7
0---93
0---9
0---91
0---90
0---47
0---56
0---36
0---23
0---63
0---18
0---27
1---4
1---6
1---7
1---94
1---93
1---92
1---11
1---22
1---76
1---72
1---38
1---54
1---29
1---68
1---53
2---3
2---5
2---99
2---6
2---98
2---7
2---97
2---96
2---9
2---95
2---10
2---11
2---12
2---23
2---32
2---21
2---82
2---90
2---87
2---48
2---73
2---51
3---7
3---99
3---9
3---94
3---13
3---93
3---34
3---46
3---15
3---43
3---54
3---82
3---63
3---28
3---8
3---67
4---5
4---6
4---9
4---10
4---98
4---11
4---97
4---12
4---96
4---13
4---95
4---76
4---43
4---66
4---89
4---58
4---92
4---46
4---75
5---7
5---8
5---10
5---98
5---13
5---97
5---96
5---15
5---69
5---17
5---54
5---88
5---35
5---68
5---11
5---70
5---56
5---32
5---51
6---7
6---8
6---9
6---10
6---11
6---12
6---99
6---14
6---15
6---97
6---96
6---46
6---30
6---38
6---79
6---31
6---50
6---13
6---59
7---8
7---10
7---14
7---97
7---29
7---39
7---58
7---91
7---93
7---31
7---65
7---43
7---38
8---11
8---15
8---17
8---18
8---67
8--

WARNING    'v' is an internal variable of group 'EG', but also exists in the run namespace with the value 99. The internal variable will be used. [brian2.groups.group.Group.resolve.resolution_conflict]
WARNING    'v' is an internal variable of group 'IG', but also exists in the run namespace with the value 99. The internal variable will be used. [brian2.groups.group.Group.resolve.resolution_conflict]
100%|██████████| 2/2 [00:00<00:00,  3.21it/s]


1.0


100%|██████████| 2/2 [00:02<00:00,  1.23s/it]

1.0


In [12]:
def test1():
    '''
    Feed train set to SNN with STDP
    Freeze STDP
    Feed train set to SNN again and collect generated features
    Train RandomForest on the top of these features and labels provided
    Feed test set to SNN and collect new features
    Predict labels with RandomForest and calculate accuacy score
    '''
    seed(0)
    
    model = Model()
    model.train(X_train, y_train, epoch=100, draw=True)
    model.net.store('train', 'train.b2')
    #model.net.restore('train', './train.b2')
    
    f_train = model.evaluate(X_test, y_test, draw=False)
    clf = RandomForestClassifier(max_depth=4, random_state=0)
    clf.fit(f_train, y_test)
    print(clf.score(f_train, y_test))

    f_test = model.evaluate(X_eval, y_eval, draw=True)
    y_pred = clf.predict(f_test)
    print(accuracy_score(y_pred, y_eval))

    cm = confusion_matrix(y_pred, y_eval)
    # print(cm)
    
    plt.figure(figsize=(10, 8))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', square=True,
                xticklabels=list(unique(y_test)),
                yticklabels=list(unique(y_test)))
    
    
    plt.ylabel('Истинные метки')
    plt.xlabel('Предсказанные метки')
    plt.title('Матрица ошибок')

    plt.show()
    
def test2(train_items=2, assign_items=2, eval_items=2):
    '''
    Feed train set to SNN with STDP
    Freeze STDP
    Feed train set to SNN again and collect generated features
    Train RandomForest on the top of these features and labels provided
    Feed test set to SNN and collect new features
    Predict labels with RandomForest and calculate accuacy score
    '''
    seed(0)
    
    model = Model()
    model.train(X_train[:train_items], y_train[:train_items], epoch=100, drow=True)
    model.net.store('train', 'train.b2')
    #model.net.restore('train', './train.b2')
    
    f_train = model.evaluate(X_train[:assign_items], y_train[:assign_items], drow=False)
    clf = RandomForestClassifier(max_depth=4, random_state=0)
    clf.fit(f_train, y_train[:assign_items])
    print(clf.score(f_train, y_train[:assign_items]))

    f_test = model.evaluate(X_test[:eval_items], y_test[:eval_items], drow=True)
    y_pred = clf.predict(f_test)
    print(accuracy_score(y_pred, y_test[:eval_items]))

    cm = confusion_matrix(y_pred, y_test[:eval_items])
    
    plt.figure(figsize=(10, 8))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', square=True,
                xticklabels=list(unique(y_test)),
                yticklabels=list(unique(y_test)))
    
    
    plt.ylabel('Истинные метки')
    plt.xlabel('Предсказанные метки')
    plt.title('Матрица ошибок')

    plt.show()
    
test1()